In [17]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.

This is by no means production code.
"""
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 51) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
# generate list of urls to visit
for page in N_PAGES:
    print(page)
    url = BASE_URL + f"/rent/melbourne-region-vic/?sort=price-desc&page={page}"
    #bs_object = BeautifulSoup(urlopen(url), "lxml")
    bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")
    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
        )
    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])

# for each url, scrape some basic metadata
for property_url in url_links:
    print(property_url)
    try:
        #bs_object = BeautifulSoup(urlopen(property_url), "lxml")
        bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

        # looks for the header class to get property name
        property_metadata[property_url]['name'] = bs_object \
            .find("h1", {"class": "css-164r41r"}) \
            .text

        # looks for the div containing a summary title for cost
        property_metadata[property_url]['cost_text'] = bs_object \
            .find("div", {"data-testid": "listing-details__summary-title"}) \
            .text
        # extract coordinates from the hyperlink provided
        # i'll let you figure out what this does :P
        property_metadata[property_url]['coordinates'] = [
            float(coord) for coord in re.findall(
                r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
                bs_object \
                    .find(
                        "a",
                        {"target": "_blank", 'rel': "noopener noreferer"}
                    ) \
                    .attrs['href']
            )[0].split(',')
        ]
        property_metadata[property_url]['rooms'] = [
            re.findall(r'\d\s[A-Za-z]+', feature.text)[0] for feature in bs_object \
                .find("div", {"data-testid": "property-features"}) \
                .findAll("span", {"data-testid": "property-features-text-container"})
        ]
        property_metadata[property_url]['desc'] = re \
            .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
            .strip('</p>')
    except:
        continue
# output to example json in data/raw/
with open('./data/raw/example.json', 'w') as f:
    dump(property_metadata, f)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
https://www.domain.com.au/202-4-chapel-mews-south-yarra-vic-3141-16065573
https://www.domain.com.au/667-glenhuntly-road-caulfield-vic-3162-11598047
https://www.domain.com.au/upstairs-2c-staley-street-brunswick-vic-3056-13168913
https://www.domain.com.au/level-3-302-13-15-lake-street-caroline-springs-vic-3023-15994395
https://www.domain.com.au/9-lansdowne-street-blairgowrie-vic-3942-12127675
https://www.domain.com.au/52-black-st-brighton-vic-3186-15410646
https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767
https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303
https://www.domain.com.au/440-keys-road-flinders-vic-3929-16065322
https://www.domain.com.au/335-the-esplanade-indented-head-vic-3223-12688424
https://www.domain.com.au/50-south-wharf-drive-docklands-vic-3008-16048359
https://www.domain.com.au/90

In [16]:
""" import json

if __name__ == '__main__':

        with open('../data/raw/example.json', 'r',encoding='utf-8') as f:
            data = json.loads(f.read())
        print(1)
        output = ''
        for obj1 in data:
            obj = data[obj1]
            try:
                print(obj)
                output += f'\n{obj["name"]},{obj["cost_text"]},{obj["coordinates"]},{obj["rooms"]},{obj["desc"]}'
            except:
                continue
        with open('./data/raw/example.csv', 'w',encoding='utf-8') as f:
            f.write(output)

 """

' import json\n\nif __name__ == \'__main__\':\n\n        with open(\'../data/raw/example.json\', \'r\',encoding=\'utf-8\') as f:\n            data = json.loads(f.read())\n        print(1)\n        output = \'\'\n        for obj1 in data:\n            obj = data[obj1]\n            try:\n                print(obj)\n                output += f\'\n{obj["name"]},{obj["cost_text"]},{obj["coordinates"]},{obj["rooms"]},{obj["desc"]}\'\n            except:\n                continue\n        with open(\'./data/raw/example.csv\', \'w\',encoding=\'utf-8\') as f:\n            f.write(output)\n\n '

In [17]:
import json
with open('../data/raw/example.json', 'r',encoding='utf-8') as f:
    data = json.loads(f.read())

JSONDecodeError: Extra data: line 1 column 4 (char 3)

In [18]:
name = []
cost_text = []
coordinate = []
rooms = []
desc = []
for obj1 in data:
    obj = data[obj1]
    try:
        name.append(obj["name"])
        cost_text.append(obj["cost_text"])
        coordinate.append(obj["coordinates"])
        rooms.append(obj["rooms"])
        desc.append(obj["desc"])
    except:
        continue

In [4]:
import pandas as pd
path_x = pd.Series(name)
path_y = pd.Series(cost_text)
path_z = pd.Series(coordinate)
path_c = pd.Series(rooms)
path_t = pd.Series(desc)
path_df = pd.DataFrame()
path_df['name'] = path_x
path_df['cost_text'] = path_y
path_df['coordinate'] = path_z
path_df['rooms'] = path_c
path_df['desc'] = path_t

filepath = "../data/raw/example.json"
path_df.to_csv(filepath, index=False, header=False)

In [5]:
import requests
from io import StringIO
import csv


class ProcessCSVPDFDOCX(object):
    def __init__(self,csv_path):
        self._csv_path = csv_path
        self._session = requests.Session()

    def read_csv(self):
        response = self._session.get(self._csv_path)
        # 将文本设置成 utf-8 的编码方式
        response.encoding = 'utf-8'
        response_text = response.text
        data_file = StringIO(response_text)
        self.dict_reader = csv.DictReader(data_file)

        print(self.dict_reader.fieldnames)
        for row in self.dict_reader:
            print(row)
    def write_csv(self,filename):
        with open(filename, 'w', newline='') as csvfile:
            titles = list(self.dict_reader.fieldnames)
            w = csv.DictWriter(csvfile,fieldnames=titles)
            for row in self.dict_reader:
                w.writerow(row)


In [6]:
url1 = 'https://data.melbourne.vic.gov.au/api/views/4w3d-fg6r/rows.csv?accessType=DOWNLOAD'
csv1 = ProcessCSVPDFDOCX(url1)
csv1.read_csv()


['station', 'the_geom', 'pids', 'he_loop', 'lift']
OrderedDict([('station', 'Alamein'), ('the_geom', 'POINT (145.07955800000002 -37.86884299999997)'), ('pids', 'No'), ('he_loop', 'No'), ('lift', 'No')])
OrderedDict([('station', 'Albion'), ('the_geom', 'POINT (144.82470999999998 -37.77765599999998)'), ('pids', 'Dot Matrix'), ('he_loop', 'No'), ('lift', 'No')])
OrderedDict([('station', 'Alphington'), ('the_geom', 'POINT (145.031251 -37.77839599999999)'), ('pids', 'Dot Matrix'), ('he_loop', 'No'), ('lift', 'No')])
OrderedDict([('station', 'Altona'), ('the_geom', 'POINT (144.830604 -37.86724899999996)'), ('pids', 'LCD'), ('he_loop', 'No'), ('lift', 'No')])
OrderedDict([('station', 'Anstey'), ('the_geom', 'POINT (144.96056099999998 -37.761897999999974)'), ('pids', 'No'), ('he_loop', 'No'), ('lift', 'No')])
OrderedDict([('station', 'Armadale'), ('the_geom', 'POINT (145.01937400000008 -37.856434999999976)'), ('pids', 'Dot Matrix'), ('he_loop', 'No'), ('lift', 'No')])
OrderedDict([('station', 

In [7]:
csv1.write_csv("dict.csv")

In [8]:
url2 = 'https://data.melbourne.vic.gov.au/api/views/i35p-5m6u/rows.csv?accessType=DOWNLOAD'
csv2 = ProcessCSVPDFDOCX(url2)
csv2.read_csv()


['the_geom', 'MCCID_STR', 'MCCID_INT', 'NAME', 'XORG', 'XSOURCE', 'XDATE', 'STOP_NO']
OrderedDict([('the_geom', 'POINT (144.96690455927876 -37.8176316450406)'), ('MCCID_STR', ''), ('MCCID_INT', '1'), ('NAME', 'Swanston Street / Flinders Street'), ('XORG', 'GIS Team'), ('XSOURCE', 'Mapbase'), ('XDATE', '20111018'), ('STOP_NO', '5')])
OrderedDict([('the_geom', 'POINT (144.964479208357 -37.818324403770184)'), ('MCCID_STR', ''), ('MCCID_INT', '2'), ('NAME', 'Elizabeth Street / Flinders Street'), ('XORG', 'GIS Team'), ('XSOURCE', 'Mapbase'), ('XDATE', '20111018'), ('STOP_NO', '4')])
OrderedDict([('the_geom', 'POINT (144.9614014008424 -37.81922319307822)'), ('MCCID_STR', ''), ('MCCID_INT', '3'), ('NAME', 'Market Street / Flinders Street'), ('XORG', 'GIS Team'), ('XSOURCE', 'Mapbase'), ('XDATE', '20111018'), ('STOP_NO', '3')])
OrderedDict([('the_geom', 'POINT (144.95786314283018 -37.82023778673241)'), ('MCCID_STR', ''), ('MCCID_INT', '4'), ('NAME', 'Melbourne Aquarium / Flinders Street'), ('X

In [9]:
url3 = 'https://data.melbourne.vic.gov.au/api/views/wuf8-susg/rows.csv?accessType=DOWNLOAD'
csv3 = ProcessCSVPDFDOCX(url2)
csv3.read_csv()



['the_geom', 'MCCID_STR', 'MCCID_INT', 'NAME', 'XORG', 'XSOURCE', 'XDATE', 'STOP_NO']
OrderedDict([('the_geom', 'POINT (144.96690455927876 -37.8176316450406)'), ('MCCID_STR', ''), ('MCCID_INT', '1'), ('NAME', 'Swanston Street / Flinders Street'), ('XORG', 'GIS Team'), ('XSOURCE', 'Mapbase'), ('XDATE', '20111018'), ('STOP_NO', '5')])
OrderedDict([('the_geom', 'POINT (144.964479208357 -37.818324403770184)'), ('MCCID_STR', ''), ('MCCID_INT', '2'), ('NAME', 'Elizabeth Street / Flinders Street'), ('XORG', 'GIS Team'), ('XSOURCE', 'Mapbase'), ('XDATE', '20111018'), ('STOP_NO', '4')])
OrderedDict([('the_geom', 'POINT (144.9614014008424 -37.81922319307822)'), ('MCCID_STR', ''), ('MCCID_INT', '3'), ('NAME', 'Market Street / Flinders Street'), ('XORG', 'GIS Team'), ('XSOURCE', 'Mapbase'), ('XDATE', '20111018'), ('STOP_NO', '3')])
OrderedDict([('the_geom', 'POINT (144.95786314283018 -37.82023778673241)'), ('MCCID_STR', ''), ('MCCID_INT', '4'), ('NAME', 'Melbourne Aquarium / Flinders Street'), ('X